# Upload Dataset

In [2]:
from SCRIPTS.A_data_collection import make_dataset_for_regression
X, y = make_dataset_for_regression(n_samples=1000, n_features=3, n_informative=3, n_targets=1, bias=0.0, effective_rank=None, tail_strength=0.5, noise=0.0, seed=2023)
print(X.shape)
print(y.shape)

(1000, 3)
(1000,)


## Data analysis
Try to understand duplicates, range of values, functional dependencies

In [5]:
import pandas as pd

df = pd.DataFrame(X)
print(df.head())

          0         1         2
0  1.259016 -1.214310 -0.216152
1 -0.887212  0.688688  1.011227
2 -0.353680  1.205251 -0.017633
3 -0.576828  0.367296  0.762756
4 -0.626545  0.669255  0.453065


In [6]:
df.dtypes

0    float64
1    float64
2    float64
dtype: object

### Single column analysis

In [9]:
distinct_values=df[0].nunique()
print(distinct_values)

1000


In [10]:
mean_values = df.mean()
std_values = df.std()

print("Mean values:")
print(mean_values)

print("Standard deviation values:")
print(std_values)


Mean values:
0   -0.018361
1   -0.037786
2    0.002820
dtype: float64
Standard deviation values:
0    0.978938
1    0.987818
2    0.979472
dtype: float64


In [16]:
import pandas as pd
import numpy as np

def inject_variable_type_issue(dataset, percentage):
    # Copy the dataset to avoid modifying the original
    polluted_dataset = dataset.copy()

    # Determine the number of features to be affected
    num_features_affected = int(len(dataset.columns) * percentage)

    # Generate random indices to select features to be affected
    random_indices = np.random.choice(len(dataset.columns), num_features_affected, replace=False)

    # Convert numeric features to categorical or boolean at random indices
    for idx in random_indices:
        column_name = dataset.columns[idx]
        if np.issubdtype(dataset[column_name].dtype, np.number):
            # Convert numeric feature to categorical or boolean
            if np.random.rand() < 0.5:
                # Convert to categorical
                polluted_dataset[column_name] = pd.Categorical(polluted_dataset[column_name])
            else:
                # Convert to boolean
                polluted_dataset[column_name] = np.random.choice([True, False], size=len(dataset))

    return polluted_dataset

polluted_dataset = inject_variable_type_issue(df, 0.1)
print(polluted_dataset.head())

          0         1      2
0  1.259016 -1.214310   True
1 -0.887212  0.688688   True
2 -0.353680  1.205251  False
3 -0.576828  0.367296   True
4 -0.626545  0.669255  False
